# Strava Workout Vector Search Demo

## Overview
This notebook demonstrates **Snowflake Cortex Search** for intelligent workout discovery using semantic search.

**Business Use Case**: Cache intelligent workout generation
- Semantically match user requests to existing workouts
- Reduce compute costs by reusing similar workouts
- Handle high-volume requests with hybrid filtering

**Key Features**:
- Semantic search (natural language understanding)
- Metadata filtering (sport type, difficulty, distance)
- Complex logic (AND/OR operators)
- AI-powered summarization

---


## Setup
Connect to the demo environment:


In [1]:
-- Set context
USE ROLE STRAVA_DEMO_ADMIN;
USE DATABASE STRAVA_DEMO_SAMPLE;
USE SCHEMA VECTOR_SEARCH_DEMO;
USE WAREHOUSE STRAVA_DEMO_WH;

-- Verify Cortex Search service
SHOW CORTEX SEARCH SERVICES;


created_on,name,database_name,schema_name,target_lag,warehouse,search_column,attribute_columns,columns,definition,comment,embedding_model,indexing_state,serving_state,source_data_num_rows,primary_key_columns,scoring_profile_count
2025-10-11T00:42:16.496Z,WORKOUT_SEARCH_SERVICE,STRAVA_DEMO_SAMPLE,VECTOR_SEARCH_DEMO,1 minute,STRAVA_DEMO_WH,EMBED_STR,"ID,SPORT_TYPE,DIFFICULTY,MOVING_TIME_SECONDS,DISTANCE_METERS,GENERATION_MODEL,WORKOUT_SOURCE,STORE_VERSION","ID,EMBED_STR,SPORT_TYPE,DIFFICULTY,MOVING_TIME_SECONDS,DISTANCE_METERS,GENERATION_MODEL,WORKOUT_SOURCE,STORE_VERSION","( SELECT ID, EMBED_STR, SPORT_TYPE, DIFFICULTY, MOVING_TIME_SECONDS, DISTANCE_METERS, GENERATION_MODEL, WORKOUT_SOURCE, STORE_VERSION FROM WORKOUTS)",,snowflake-arctic-embed-m-v1.5,SUSPENDED,ACTIVE,2000,,0


---

# Demo Query 1: Basic Semantic Search

**Scenario**: "Show me endurance training workouts"

**What This Demonstrates**:
- Pure semantic search without filters
- Natural language understanding
- Relevance ranking
- Simple parameterization

**Try modifying**: Change the `query_text` or `result_limit` values below


In [1]:
-- Query 1 Parameters (modify these values)
SET query_text = 'endurance long distance training workout';
SET result_limit = 5;

-- Build the search JSON from parameters
SET search_json = (
    SELECT CONCAT(
        '{"query": "', $query_text, '",',
        ' "columns": ["ID", "EMBED_STR", "SPORT_TYPE", "DIFFICULTY", "DISTANCE_METERS", "MOVING_TIME_SECONDS"],',
        ' "limit": ', $result_limit, '}'
    )
);

-- Execute Query 1
SELECT 
    result.value:ID::VARCHAR AS ID,
    LEFT(result.value:EMBED_STR::VARCHAR, 150) AS WORKOUT_PREVIEW,
    result.value:SPORT_TYPE::VARCHAR AS SPORT_TYPE,
    result.value:DIFFICULTY::VARCHAR AS DIFFICULTY,
    ROUND(NULLIF(result.value:DISTANCE_METERS::VARCHAR, '')::FLOAT / 1000.0, 1) AS DISTANCE_KM,
    ROUND(NULLIF(result.value:MOVING_TIME_SECONDS::VARCHAR, '')::FLOAT / 60.0, 0) AS DURATION_MIN
FROM TABLE(FLATTEN(
    PARSE_JSON(
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'STRAVA_DEMO_SAMPLE.VECTOR_SEARCH_DEMO.WORKOUT_SEARCH_SERVICE',
            $search_json
        )
    )['results']
)) result;


ID,WORKOUT_PREVIEW,SPORT_TYPE,DIFFICULTY,DISTANCE_KM,DURATION_MIN
SLAM_82246cb285d95fb1,Hard Steady State## Warm-up• 1500m easy build at <PACE_ZONE_1_RECOVERY> to <PACE_ZONE_2_ENDURANCE>## Main Set• 7000m steady state at <PACE_ZONE_4,run,moderate,31.8,
SLAM_3037643ec2bd5d76,Hard Steady State## Warm-up• 1500m easy build at <PACE_ZONE_1_RECOVERY> to <PACE_ZONE_2_ENDURANCE>## Main Set• 7000m steady state at <PACE_ZONE_4,run,moderate,32.2,
SLAM_4da2a32af816ef91,Hard Steady State## Warm-up• 1500m easy build at <PACE_ZONE_1_RECOVERY> to <PACE_ZONE_2_ENDURANCE>## Main Set• 7000m steady state at <PACE_ZONE_4,run,moderate,34.9,
SLAM_a672bbbca5396f7a,Hard Steady State## Warm-up• 1500m easy build at <PACE_ZONE_1_RECOVERY> to <PACE_ZONE_2_ENDURANCE>## Main Set• 7000m steady state at <PACE_ZONE_4,run,hard,40.4,
SLAM_6597e5789f9fab3f,Hard Steady State## Warm-up• 1500m easy build at <PACE_ZONE_1_RECOVERY> to <PACE_ZONE_2_ENDURANCE>## Main Set• 7000m steady state at <PACE_ZONE_4,run,hard,37.4,


**Key Insight**: The search finds relevant workouts using semantic understanding - it matches intent, not just keywords.

---


# Demo Query 2: Hybrid Search with Filters

**Scenario**: "Find beginner-friendly 5K running workouts"

**What This Demonstrates**:
- Semantic search + metadata filtering
- Distance range filtering (4.5-6.0 km ≈ 5K)
- Difficulty level filtering
- Sport type filtering

**Try modifying**: Change `"query"`, `"SPORT_TYPE"`, `"DIFFICULTY"`, or distance ranges


In [1]:
SELECT 
    result.value:ID::VARCHAR AS ID,
    LEFT(result.value:EMBED_STR::VARCHAR, 100) AS WORKOUT_PREVIEW,
    ROUND(NULLIF(result.value:DISTANCE_METERS::VARCHAR, '')::FLOAT / 1000.0, 1) AS DISTANCE_KM,
    ROUND(NULLIF(result.value:MOVING_TIME_SECONDS::VARCHAR, '')::FLOAT / 60.0, 0) AS DURATION_MIN,
    result.value:DIFFICULTY::VARCHAR AS DIFFICULTY
FROM TABLE(FLATTEN(
    PARSE_JSON(
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'STRAVA_DEMO_SAMPLE.VECTOR_SEARCH_DEMO.WORKOUT_SEARCH_SERVICE',
            '{
                "query": "beginner friendly easy paced 5k running workout",
                "columns": ["ID", "EMBED_STR", "DISTANCE_METERS", "MOVING_TIME_SECONDS", "DIFFICULTY"],
                "filter": {"@and": [
                    {"@eq": {"SPORT_TYPE": "run"}},
                    {"@eq": {"DIFFICULTY": "easy"}},
                    {"@gte": {"DISTANCE_METERS": 4500}},
                    {"@lte": {"DISTANCE_METERS": 6000}}
                ]},
                "limit": 5
            }'
        )
    )['results']
)) result;


ID,WORKOUT_PREVIEW,DISTANCE_KM,DURATION_MIN,DIFFICULTY
SLAM_fffcde1db58ed04b,Easy Recovery Run## Warm-up• 5 min easy jog at <PACE_ZONE_1_RECOVERY>• Focus on relaxed form and,4.7,40,easy
SLAM_7726f1a625864d83,Easy Recovery Run## Warm-up• 5 min easy jog at <PACE_ZONE_1_RECOVERY>• Focus on relaxed form and,5.5,40,easy
SLAM_9ff71481b9949ee9,Easy Recovery Run## Warm-up• 5 min easy jog at <PACE_ZONE_1_RECOVERY>• Focus on relaxed form and,5.6,40,easy
SLAM_e44e667406f53c92,Easy Recovery Run## Warm-up• 5 min easy jog at <PACE_ZONE_1_RECOVERY>• Focus on relaxed form and,5.9,40,easy
SLAM_57df8e55f986c74f,Easy Recovery Run## Warm-up• 5 min easy jog at <PACE_ZONE_1_RECOVERY>• Focus on relaxed form and,4.8,40,easy


**Key Insight**: Filters ensure ALL results match criteria (sport, difficulty, distance), while semantic ranking orders by relevance.

---


# Demo Query 3: Advanced Filtering with OR Logic

**Scenario**: "High-intensity cycling for experienced athletes"

**What This Demonstrates**:
- Complex `@or` logic for multiple difficulty levels
- Filtering by sport type
- Multiple conditions with `@and` + `@or`

**Try modifying**: Change difficulty levels or sport type


In [1]:
SELECT 
    result.value:ID::VARCHAR AS ID,
    LEFT(result.value:EMBED_STR::VARCHAR, 120) AS WORKOUT_PREVIEW,
    result.value:DIFFICULTY::VARCHAR AS DIFFICULTY,
    ROUND(NULLIF(result.value:DISTANCE_METERS::VARCHAR, '')::FLOAT / 1000.0, 1) AS DISTANCE_KM,
    ROUND(NULLIF(result.value:MOVING_TIME_SECONDS::VARCHAR, '')::FLOAT / 60.0, 0) AS DURATION_MIN
FROM TABLE(FLATTEN(
    PARSE_JSON(
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'STRAVA_DEMO_SAMPLE.VECTOR_SEARCH_DEMO.WORKOUT_SEARCH_SERVICE',
            '{
                "query": "intense hard challenging cycling interval training power",
                "columns": ["ID", "EMBED_STR", "DIFFICULTY", "DISTANCE_METERS", "MOVING_TIME_SECONDS"],
                "filter": {"@and": [
                    {"@eq": {"SPORT_TYPE": "ride"}},
                    {"@or": [
                        {"@eq": {"DIFFICULTY": "hard"}},
                        {"@eq": {"DIFFICULTY": "very_hard"}}
                    ]}
                ]},
                "limit": 5
            }'
        )
    )['results']
)) result;


ID,WORKOUT_PREVIEW,DIFFICULTY,DISTANCE_KM,DURATION_MIN
SLAM_e46a83ee904b545e,Steady Zone Ride## Warm-up• 10 min gradual build from <POWER_ZONE_1_RECOVERY> to <POWER_ZONE_2_ENDURANCE>• Focus on s,hard,,73
SLAM_3651aa26826074b7,Steady Zone Ride## Warm-up• 10 min gradual build from <POWER_ZONE_1_RECOVERY> to <POWER_ZONE_2_ENDURANCE>• Focus on s,hard,,74
SLAM_1b3058a10bdd3c3f,Steady Zone Ride## Warm-up• 10 min gradual build from <POWER_ZONE_1_RECOVERY> to <POWER_ZONE_2_ENDURANCE>• Focus on s,hard,,109
SLAM_be3ac8a45bbe0eec,Steady Zone Ride## Warm-up• 10 min gradual build from <POWER_ZONE_1_RECOVERY> to <POWER_ZONE_2_ENDURANCE>• Focus on s,hard,,119
SLAM_d857f429b673bb57,Steady Zone Ride## Warm-up• 10 min gradual build from <POWER_ZONE_1_RECOVERY> to <POWER_ZONE_2_ENDURANCE>• Focus on s,very_hard,,128


**Key Insight**: Complex `@or` logic enables flexible filtering - returns workouts matching ANY of the specified difficulty levels.

---


# Demo Query 4: Intelligent Workout Modification

**Scenario**: "I found a workout but have constraints - modify it for me"

**What This Demonstrates**:
- AI-powered workout adaptation
- Real-world personalization (time constraints, injuries, equipment)
- Maintains training intent while adjusting execution

**Try modifying**: Change the constraints (time limit, injury type, equipment availability)


In [1]:
# Query 4: Intelligent Workout Modification
# Find a workout and modify it for constraints (30 min + knee pain)

# Get Snowflake session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Step 1: Search for a threshold workout
search_query = """
SELECT 
    result.value:ID::VARCHAR AS WORKOUT_ID,
    result.value:EMBED_STR::VARCHAR AS WORKOUT_TEXT,
    result.value:DIFFICULTY::VARCHAR AS DIFFICULTY,
    ROUND(TRY_CAST(result.value:DISTANCE_METERS::VARCHAR AS INT) / 1000.0, 1) AS DISTANCE_KM
FROM TABLE(FLATTEN(
    PARSE_JSON(
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'STRAVA_DEMO_SAMPLE.VECTOR_SEARCH_DEMO.WORKOUT_SEARCH_SERVICE',
            '{
                "query": "threshold tempo interval training",
                "columns": ["ID", "EMBED_STR", "DIFFICULTY", "DISTANCE_METERS"],
                "filter": {"@eq": {"SPORT_TYPE": "run"}},
                "limit": 1
            }'
        )
    )['results']
)) result
"""

original_workout = session.sql(search_query).collect()[0]

# Step 2: Use AI to modify the workout
modification_prompt = f"""Modify this workout for an athlete with only 30 minutes available and recovering from knee pain.
Keep the same training intent (threshold/tempo work) but adjust volume and impact.
Format your response with clear sections: Warm-up, Main Set, Cool-down, and Training Notes.

Original workout:
{original_workout['WORKOUT_TEXT']}
"""

# Escape single quotes for SQL
escaped_prompt = modification_prompt.replace("'", "''")

ai_query = f"""
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'mistral-large2',
    '{escaped_prompt}'
) AS MODIFIED_WORKOUT
"""

modified = session.sql(ai_query).collect()[0]

# Step 3: Display results in a nice format
print("="*80)
print("INTELLIGENT WORKOUT MODIFICATION")
print("="*80)
print()
print("ORIGINAL WORKOUT")
print("-"*80)
print(f"ID: {original_workout['WORKOUT_ID']}")
print(f"Difficulty: {original_workout['DIFFICULTY']}")
print(f"Distance: {original_workout['DISTANCE_KM']} km")
print()
print(original_workout['WORKOUT_TEXT'][:300] + "...")
print()
print("="*80)
print("MODIFIED WORKOUT (30 min, knee-friendly)")
print("="*80)
print()
print(modified['MODIFIED_WORKOUT'])
print()
print("="*80)


INTELLIGENT WORKOUT MODIFICATION  
ORIGINAL WORKOUT  
--------------------------------------------------------------------------------  
ID: SLAM_17404d40edd287f6  
Difficulty: moderate  
Distance: 4.5 km  
5K Threshold Intervals  
## Warm-up  
• 1000m easy at <PACE_ZONE_2_ENDURANCE>  
## Main Set  
### Threshold Intervals  
4 intervals of:  
• 800m at <PACE_ZONE_4_THRESHOLD> (comfortably hard, sustainable effort)  
• 200m recovery jog at <PACE_ZONE_1_RECOVERY>  
## Cool-down  
• 800m easy at <PACE_ZONE_2_ENDURANCE>  
...  
MODIFIED WORKOUT (30 min, knee-friendly)  
 ### Modified Workout for Athlete with 30 Minutes and Recovering from Knee Pain  
**Training Intent:** Threshold/Tempo work with reduced volume and impact.  
---  
### Warm-up  
- **5 minutes** of easy walking or slow jogging at <PACE_ZONE_2_ENDURANCE>  
- **2 minutes** of dynamic stretching (leg swings, hip circles, etc.)  
---  
### Main Set  
#### Threshold Intervals  
3 intervals of:  
- **400m** at <PACE_ZONE_4_THRESHO

**Key Insight**: AI can intelligently adapt workouts based on real-world constraints while maintaining training principles.

---

# Demo Query 5: Progressive Workout Recommendation

**Scenario**: "I just completed this 5K workout - give me a slightly harder one to progress"

**What This Demonstrates**:
- Conversational AI with workout context
- Progressive training logic (gradual overload)
- Understands workout characteristics and progression
- Personalized next-step recommendations

**Try modifying**: Change the completed workout or progression request (easier, different focus, etc.)


In [1]:
# Query 5: Progressive Workout Recommendation
# User completed a workout and wants a harder progression

# Get Snowflake session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Step 1: Get the workout user just completed (simulating user input)
# Use Cortex Search to find a moderate 5K-style workout
completed_search = """
SELECT 
    result.value:ID::VARCHAR AS ID,
    result.value:EMBED_STR::VARCHAR AS EMBED_STR,
    result.value:SPORT_TYPE::VARCHAR AS SPORT_TYPE,
    result.value:DIFFICULTY::VARCHAR AS DIFFICULTY,
    TRY_CAST(result.value:DISTANCE_METERS::VARCHAR AS INT) AS DISTANCE_METERS,
    TRY_CAST(result.value:MOVING_TIME_SECONDS::VARCHAR AS INT) AS MOVING_TIME_SECONDS
FROM TABLE(FLATTEN(
    PARSE_JSON(
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'STRAVA_DEMO_SAMPLE.VECTOR_SEARCH_DEMO.WORKOUT_SEARCH_SERVICE',
            '{
                "query": "5K tempo moderate running workout",
                "columns": ["ID", "EMBED_STR", "SPORT_TYPE", "DIFFICULTY", "DISTANCE_METERS", "MOVING_TIME_SECONDS"],
                "filter": {"@and": [
                    {"@eq": {"SPORT_TYPE": "run"}},
                    {"@eq": {"DIFFICULTY": "moderate"}}
                ]},
                "limit": 1
            }'
        )
    )['results']
)) result
"""

completed_results = session.sql(completed_search).collect()

if not completed_results:
    print("ERROR: No moderate running workouts found in database")
    print("Please ensure workouts are loaded properly.")
else:
    completed = completed_results[0]
    
    # Step 2: Search for harder workouts in the same domain
    progression_search = """
    SELECT 
        result.value:ID::VARCHAR AS WORKOUT_ID,
        result.value:EMBED_STR::VARCHAR AS WORKOUT_TEXT,
        result.value:DIFFICULTY::VARCHAR AS DIFFICULTY,
        ROUND(TRY_CAST(result.value:DISTANCE_METERS::VARCHAR AS INT) / 1000.0, 1) AS DISTANCE_KM,
        ROUND(TRY_CAST(result.value:MOVING_TIME_SECONDS::VARCHAR AS INT) / 60.0, 0) AS DURATION_MIN
    FROM TABLE(FLATTEN(
        PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'STRAVA_DEMO_SAMPLE.VECTOR_SEARCH_DEMO.WORKOUT_SEARCH_SERVICE',
                '{
                    "query": "5K interval threshold running workout",
                    "columns": ["ID", "EMBED_STR", "DIFFICULTY", "DISTANCE_METERS", "MOVING_TIME_SECONDS"],
                    "filter": {"@and": [
                        {"@eq": {"SPORT_TYPE": "run"}},
                        {"@or": [
                            {"@eq": {"DIFFICULTY": "hard"}},
                            {"@eq": {"DIFFICULTY": "very hard"}}
                        ]}
                    ]},
                    "limit": 3
                }'
            )
        )['results']
    )) result
    """
    
    candidates = session.sql(progression_search).collect()
    
    if not candidates:
        print("ERROR: No harder workouts found")
    else:
        # Step 3: Use AI to recommend the best progression
        candidates_summary = "\n".join([
            f"Option {i+1} ({c['DIFFICULTY']}): {c['WORKOUT_TEXT'][:200]}"
            for i, c in enumerate(candidates)
        ])
        
        recommendation_prompt = f"""You are a running coach helping an athlete progress their training.

The athlete just completed this workout:
DIFFICULTY: {completed['DIFFICULTY']}
WORKOUT: {completed['EMBED_STR'][:300]}

They want a slightly harder workout to continue improving. Here are {len(candidates)} harder workout options:

{candidates_summary}

Your task:
1. Analyze the completed workout's characteristics (pace zones, intervals, volume)
2. Review the {len(candidates)} harder options
3. Recommend the BEST next workout for progressive overload
4. Explain WHY this is the right progression (what makes it harder but appropriate)
5. Give 1-2 tips for successfully completing the harder workout

Format: 
Recommendation: [Option number and name]
Why: [Explanation]
Tips: [Advice]
"""
        
        # Escape single quotes for SQL
        escaped_prompt = recommendation_prompt.replace("'", "''")
        
        ai_query = f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2',
            '{escaped_prompt}'
        ) AS RECOMMENDATION
        """
        
        recommendation = session.sql(ai_query).collect()[0]
        
        # Step 4: Display results with rich formatting
        print("="*80)
        print("PROGRESSIVE WORKOUT RECOMMENDATION")
        print("="*80)
        print()
        print("USER REQUEST: 'I just did the 5K workout - give me something harder'")
        print()
        print("-"*80)
        print("COMPLETED WORKOUT")
        print("-"*80)
        print(f"Difficulty: {completed['DIFFICULTY']}")
        if completed['DISTANCE_METERS']:
            print(f"Distance: {completed['DISTANCE_METERS']/1000:.1f} km")
        print()
        print(completed['EMBED_STR'][:250])
        print("...")
        print()
        print("="*80)
        print("PROGRESSION OPTIONS FOUND")
        print("="*80)
        print()
        for i, candidate in enumerate(candidates, 1):
            print(f"Option {i}: [{candidate['DIFFICULTY']:10s}] {candidate['DISTANCE_KM']} km, {candidate['DURATION_MIN']} min")
            print(f"  {candidate['WORKOUT_TEXT'].split(chr(10))[0]}")
            print()
        
        print("="*80)
        print("AI COACH RECOMMENDATION")
        print("="*80)
        print()
        print(recommendation['RECOMMENDATION'])
        print()
        print("="*80)


PROGRESSIVE WORKOUT RECOMMENDATION  
USER REQUEST: 'I just did the 5K workout - give me something harder'  
--------------------------------------------------------------------------------  
COMPLETED WORKOUT  
--------------------------------------------------------------------------------  
Difficulty: moderate  
Distance: 4.6 km  
5K Threshold Intervals  
## Warm-up  
• 1000m easy at <PACE_ZONE_2_ENDURANCE>  
## Main Set  
### Threshold Intervals  
4 intervals of:  
• 800m at <PACE_ZONE_4_THRESHOLD> (comfortably hard, sustainable effort)  
• 200m recovery jog at <PACE_ZONE_1_RECOVERY>  
##   
...  
PROGRESSION OPTIONS FOUND  
Option 1: [hard      ] 5.5 km, None min  
  5K Threshold Intervals  
Option 2: [hard      ] 5.5 km, None min  
  5K Threshold Intervals  
Option 3: [hard      ] 4.6 km, None min  
  5K Threshold Intervals  
AI COACH RECOMMENDATION  
 Recommendation: Option 1 (hard): 5K Threshold Intervals  
Why: Option 1 is the best next workout for progressive overload because

**Key Insight**: AI understands training progression principles and can recommend appropriate next steps based on completed workouts - simulating a personal coach.

---
